In [1]:
code = '000660'          

tr_name = 'TR_SCHART'  

start_date = '20100101'   

end_date = '20200229' 

Lookup = '9999' 

In [2]:
url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=D&2=1' +\
      '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

In [3]:
from urllib.request import urlopen
url_page = urlopen(url)

In [4]:
import json
url_data = json.loads(url_page.read())

In [5]:
import pandas as pd
data = pd.DataFrame(url_data)

In [6]:
data.columns = ['일자','시간','시가','고가','저가','종가','주가수정계수','거래량수정계수','락구분','단위거래량','단위거래대금']
data = data[['일자', '시간', '시가', '고가', '저가', '종가', '단위거래량', '단위거래대금']]
data = data[::-1]
data = data.set_index('일자')
data = data[['시가', '고가', '저가', '종가', '단위거래량', '단위거래대금']]

In [7]:
data

,시가,고가,저가,종가,단위거래량,단위거래대금
일자,,,,,,
20100104,23350,24150,23350,24100,7346425,17569735945
20100105,24500,24900,23050,23350,12118281,28970217040
20100106,23700,24550,23600,24550,7731186,18627220450
20100107,25000,25200,24350,24400,11084814,27488432680
20100108,24650,24850,23450,24650,9695685,23401422925
...,...,...,...,...,...,...
20200224,100500,100500,98900,99500,5550758,55310521575
20200225,97800,99100,96400,98000,6594059,64288585365
20200226,95500,96100,94400,94600,6405409,60767436220


In [8]:
import numpy as np
for column in data.columns : 
    data[column] = data[column].apply(lambda x : np.int(x))

In [9]:
data['내일 종가 변화율'] = 100 * (data['종가'].shift(-1) - data['종가']) / data['종가']
data['내일 종가 변화율'] = round(data['내일 종가 변화율'], 4)

In [10]:
data['시가 대비 고가 위치'] = 100 * (data['고가'] - data['시가']) / data['시가']
data['시가 대비 고가 위치'] = round(data['시가 대비 고가 위치'], 4)

In [11]:
data['시가 대비 저가 위치'] = 100 * (data['저가'] - data['시가']) / data['시가']
data['시가 대비 저가 위치'] = round(data['시가 대비 저가 위치'], 4)
data['시가 대비 종가 위치'] = 100 * (data['종가'] - data['시가']) / data['시가']
data['시가 대비 종가 위치'] = round(data['시가 대비 종가 위치'], 4)

In [12]:
data['저가 대비 고가 위치'] = 100 * (data['고가'] - data['저가']) / data['저가']
data['저가 대비 고가 위치'] = round(data['저가 대비 고가 위치'], 4)
data['저가 대비 종가 위치'] = 100 * (data['종가'] - data['저가']) / data['저가']
data['저가 대비 종가 위치'] = round(data['저가 대비 종가 위치'], 4)

In [13]:
data['고가 대비 종가 위치'] = 100 * (data['종가'] - data['고가']) / data['고가']
data['고가 대비 종가 위치'] = round(data['고가 대비 종가 위치'], 4)

In [14]:
data['root(log(단위거래량))'] = data['단위거래량'].apply(lambda x : np.sqrt(np.log(x+1)))
data['root(log(단위거래대금))'] = data['단위거래대금'].apply(lambda x : np.sqrt(np.log(x+1)))

In [15]:
data = data[data['단위거래량'] != 0]
data = data[data['단위거래대금'] != 0]

In [16]:
data['1일전 변화율'] = 0.95 * data['내일 종가 변화율'].shift(1)
data['2일전 변화율'] = 0.90 * data['내일 종가 변화율'].shift(2)
data['3일전 변화율'] = 0.85 * data['내일 종가 변화율'].shift(3)
data['4일전 변화율'] = 0.80 * data['내일 종가 변화율'].shift(4)
data['5일전 변화율'] = 0.75 * data['내일 종가 변화율'].shift(5)

In [17]:
data

,시가,고가,저가,종가,단위거래량,단위거래대금,내일 종가 변화율,시가 대비 고가 위치,시가 대비 저가 위치,시가 대비 종가 위치,저가 대비 고가 위치,저가 대비 종가 위치,고가 대비 종가 위치,root(log(단위거래량)),root(log(단위거래대금)),1일전 변화율,2일전 변화율,3일전 변화율,4일전 변화율,5일전 변화율
일자,,,,,,,,,,,,,,,,,,,,
20100104,23350,24150,23350,24100,7346425,17569735945,-3.1120,3.4261,0.0000,3.2120,3.4261,3.2120,-0.2070,3.976144,4.856897,NaN,NaN,NaN,NaN,NaN
20100105,24500,24900,23050,23350,12118281,28970217040,5.1392,1.6327,-5.9184,-4.6939,8.0260,1.3015,-6.2249,4.038592,4.908109,-2.956400,NaN,NaN,NaN,NaN
20100106,23700,24550,23600,24550,7731186,18627220450,-0.6110,3.5865,-0.4219,3.5865,4.0254,4.0254,0.0000,3.982559,4.862910,4.882240,-2.80080,NaN,NaN,NaN
20100107,25000,25200,24350,24400,11084814,27488432680,1.0246,0.8000,-2.6000,-2.4000,3.4908,0.2053,-3.1746,4.027541,4.902758,-0.580450,4.62528,-2.645200,NaN,NaN
20100108,24650,24850,23450,24650,9695685,23401422925,-2.6369,0.8114,-4.8682,0.0000,5.9701,5.1173,-0.8048,4.010884,4.886314,0.973370,-0.54990,4.368320,-2.48960,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200224,100500,100500,98900,99500,5550758,55310521575,-1.5075,0.0000,-1.5920,-0.9950,1.6178,0.6067,-0.9950,3.940742,4.973553,-3.228195,-0.86535,0.410635,1.17648,-2.142825
20200225,97800,99100,96400,98000,6594059,64288585365,-3.4694,1.3292,-1.4315,0.2045,2.8008,1.6598,-1.1100,3.962535,4.988652,-1.432125,-3.05829,-0.817275,0.38648,1.102950
20200226,95500,96100,94400,94600,6405409,60767436220,-1.9027,0.6283,-1.1518,-0.9424,1.8008,0.2119,-1.5609,3.958870,4.983003,-3.295930,-1.35675,-2.888385,-0.76920,0.362325


In [18]:
data = data[['내일 종가 변화율','시가 대비 고가 위치',
             '시가 대비 저가 위치', '시가 대비 종가 위치', '저가 대비 고가 위치', '저가 대비 종가 위치',
             '고가 대비 종가 위치', 'root(log(단위거래량))', 'root(log(단위거래대금))', '1일전 변화율',
             '2일전 변화율', '3일전 변화율', '4일전 변화율', '5일전 변화율']]

In [19]:
data = data.dropna()

In [20]:
data

,내일 종가 변화율,시가 대비 고가 위치,시가 대비 저가 위치,시가 대비 종가 위치,저가 대비 고가 위치,저가 대비 종가 위치,고가 대비 종가 위치,root(log(단위거래량)),root(log(단위거래대금)),1일전 변화율,2일전 변화율,3일전 변화율,4일전 변화율,5일전 변화율
일자,,,,,,,,,,,,,,
20100111,0.2083,0.4016,-4.6185,-3.6145,5.2632,1.0526,-4.0000,3.965492,4.849819,-2.505055,0.92214,-0.519350,4.11136,-2.334000
20100112,0.0000,1.8634,-1.4493,-0.4141,3.3613,1.0504,-2.2358,3.917300,4.809904,0.197885,-2.37321,0.870910,-0.48880,3.854400
20100113,5.8212,2.9412,-0.4202,1.0504,3.3755,1.4768,-1.8367,3.927690,4.818139,0.000000,0.18747,-2.241365,0.81968,-0.458250
20100114,2.5540,4.5082,-0.4098,4.3033,4.9383,4.7325,-0.1961,4.032167,4.907710,5.530140,0.00000,0.177055,-2.10952,0.768450
20100115,-0.7663,3.2882,-0.5803,0.9671,3.8911,1.5564,-2.2472,4.035282,4.914643,2.426300,5.23908,0.000000,0.16664,-1.977675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200221,-3.3981,1.9704,-0.4926,1.4778,2.4752,1.9802,-0.4831,3.901007,4.945317,-0.913425,0.43479,1.250010,-2.28568,0.358875
20200224,-1.5075,0.0000,-1.5920,-0.9950,1.6178,0.6067,-0.9950,3.940742,4.973553,-3.228195,-0.86535,0.410635,1.17648,-2.142825
20200225,-3.4694,1.3292,-1.4315,0.2045,2.8008,1.6598,-1.1100,3.962535,4.988652,-1.432125,-3.05829,-0.817275,0.38648,1.102950


In [21]:
X = data[data.columns[1:]]
y = data[data.columns[0]]

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1234)

In [23]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=6, 
                             max_features=10, 
                             n_estimators=30,
                             min_samples_leaf=10,
                             min_samples_split=10,
                             n_jobs=-1, 
                             random_state=1234)
regr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features=10, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=10,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=30, n_jobs=-1, oob_score=False,
                      random_state=1234, verbose=0, warm_start=False)

In [24]:
result = pd.DataFrame({'y_true':y_train, 'y_pred':regr.predict(X_train)})
result['y_pred_UD'] = result['y_pred'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
result['y_true_UD'] = result['y_true'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
result

,y_true,y_pred,y_pred_UD,y_true_UD
일자,,,,
20130626,1.3223,-0.175102,DOWN,UP
20180220,1.8543,0.267956,UP,UP
20130328,-1.7094,-0.085582,DOWN,DOWN
20130716,0.5272,0.144490,UP,UP
20180518,1.3652,0.112564,UP,UP
...,...,...,...,...
20110222,-4.1667,-0.391236,DOWN,DOWN
20180413,-0.2375,-0.085214,DOWN,DOWN
20120906,7.6555,0.686400,UP,UP


In [25]:
table_train = pd.crosstab(result.y_pred_UD, result.y_true_UD, margins=True)
table_train

y_true_UD,DOWN,UP,All
y_pred_UD,,,
DOWN,599,224,823
UP,350,702,1052
All,949,926,1875


In [26]:
round((np.array(table_train)[0,0] + np.array(table_train)[1,1]) / np.array(table_train)[2,2], 4)

0.6939

In [27]:
result = pd.DataFrame({'y_true':y_test, 'y_pred':regr.predict(X_test)})
result['y_pred_UD'] = result['y_pred'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
result['y_true_UD'] = result['y_true'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
result

,y_true,y_pred,y_pred_UD,y_true_UD
일자,,,,
20130823,-0.5263,0.226752,UP,DOWN
20120920,-0.2092,0.661784,UP,DOWN
20160801,-3.1564,0.127966,UP,DOWN
20141202,0.7338,0.060276,UP,UP
20181018,3.0568,0.235275,UP,UP
...,...,...,...,...
20110727,-1.0504,-0.185108,DOWN,DOWN
20120410,-1.9820,0.042913,UP,DOWN
20160119,-5.3004,0.341869,UP,DOWN


In [28]:
table_test = pd.crosstab(result.y_pred_UD, result.y_true_UD, margins=True)
table_test

y_true_UD,DOWN,UP,All
y_pred_UD,,,
DOWN,141,131,272
UP,167,186,353
All,308,317,625


In [29]:
round((np.array(table_test)[0,0] + np.array(table_test)[1,1]) / np.array(table_test)[2,2], 4)

0.5232

In [30]:
start_date = '20200301'   

end_date = '20200312' 

Lookup = '9999' 

url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=D&2=1' +\
      '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

from urllib.request import urlopen
url_page = urlopen(url)

import json
url_data = json.loads(url_page.read())

import pandas as pd
data = pd.DataFrame(url_data)

data.columns = ['일자','시간','시가','고가','저가','종가','주가수정계수','거래량수정계수','락구분','단위거래량','단위거래대금']
data = data[['일자', '시간', '시가', '고가', '저가', '종가', '단위거래량', '단위거래대금']]
data = data[::-1]
data = data.set_index('일자')
data = data[['시가', '고가', '저가', '종가', '단위거래량', '단위거래대금']]

import numpy as np
for column in data.columns : 
    data[column] = data[column].apply(lambda x : np.int(x))
    
data['내일 종가 변화율'] = 100 * (data['종가'].shift(-1) - data['종가']) / data['종가']
data['내일 종가 변화율'] = round(data['내일 종가 변화율'], 4)

data['시가 대비 고가 위치'] = 100 * (data['고가'] - data['시가']) / data['시가']
data['시가 대비 고가 위치'] = round(data['시가 대비 고가 위치'], 4)

data['시가 대비 저가 위치'] = 100 * (data['저가'] - data['시가']) / data['시가']
data['시가 대비 저가 위치'] = round(data['시가 대비 저가 위치'], 4)
data['시가 대비 종가 위치'] = 100 * (data['종가'] - data['시가']) / data['시가']
data['시가 대비 종가 위치'] = round(data['시가 대비 종가 위치'], 4)

data['저가 대비 고가 위치'] = 100 * (data['고가'] - data['저가']) / data['저가']
data['저가 대비 고가 위치'] = round(data['저가 대비 고가 위치'], 4)
data['저가 대비 종가 위치'] = 100 * (data['종가'] - data['저가']) / data['저가']
data['저가 대비 종가 위치'] = round(data['저가 대비 종가 위치'], 4)

data['고가 대비 종가 위치'] = 100 * (data['종가'] - data['고가']) / data['고가']
data['고가 대비 종가 위치'] = round(data['고가 대비 종가 위치'], 4)

data['root(log(단위거래량))'] = data['단위거래량'].apply(lambda x : np.sqrt(np.log(x+1)))
data['root(log(단위거래대금))'] = data['단위거래대금'].apply(lambda x : np.sqrt(np.log(x+1)))

data = data[data['단위거래량'] != 0]
data = data[data['단위거래대금'] != 0]

data['1일전 변화율'] = 0.95 * data['내일 종가 변화율'].shift(1)
data['2일전 변화율'] = 0.90 * data['내일 종가 변화율'].shift(2)
data['3일전 변화율'] = 0.85 * data['내일 종가 변화율'].shift(3)
data['4일전 변화율'] = 0.80 * data['내일 종가 변화율'].shift(4)
data['5일전 변화율'] = 0.75 * data['내일 종가 변화율'].shift(5)

data = data.dropna()

data_2 = data

data = data[data.columns[7:]]

# Predict rate

ccc = 100 * (np.array(data_2['종가'].shift(-1)) - np.array(data_2['종가'])) / np.array(data_2['종가'])

# DataFrame Result
predict = pd.DataFrame({'오늘 실제 종가' : np.array(data_2['종가']),
                        '내일 실제 종가' : np.array(data_2['종가'].shift(-1)),
                        '실제 내일 변동(%)' : ccc,
                        '모델 예상 내일 변동(%)' : regr.predict(data)})
# Result
round(predict, 2)

,오늘 실제 종가,내일 실제 종가,실제 내일 변동(%),모델 예상 내일 변동(%)
0,86900,89100.0,2.53,0.29
1,89100,85500.0,-4.04,-0.21
2,85500,NaN,NaN,1.09
